# Image Classification: More Pets
 Trains a model to classify an image as a cat, dog, rabbit, mouse, hamster, or fish.

Below we do the following:

1. Setup training environment.
2. Load images of cats, dogs, rabbits, mic, hamsters, and fish.
3. Train an image classifier model.
4. Convert the model to CoreML format and download it.

## Environment Setup

Below we ensure `CUDA 10` is installed and then use pip to install `turicreate` and `mxnet-cu100` libraries.

In [0]:
# Confirm that you have CUDA 10
!nvcc --version

In [0]:
# Install libraries - you will need to restart the runtime after doing this
!pip install turicreate
# The wrong version of MXNET will be installed
!pip uninstall -y mxnet
# Install CUDA10-compatible version of mxnet-cuXX
!pip install mxnet-cu100

## Data Preparation and Model Training

In [0]:
# Import necessary libraries - this may throw warnings
import urllib
import tarfile

import coremltools
import turicreate as tc

In [0]:
# Specify the data set download url
data_url = "https://s3.amazonaws.com/skafos.example.data/ImageClassifier/MorePets.tar.gz"
data_path = "MorePets.tar.gz"

# Pull the compressed data and extract it
retrieve = urllib.request.urlretrieve(data_url, data_path)
tar = tarfile.open(data_path)
tar.extractall()
tar.close()

In [0]:
# Load images - you can ignore various jpeg decode errors
data = tc.image_analysis.load_images('MorePets', with_path=True, ignore_failure=True)

# From the path-name, create a label column. This labels each image as the appropriate animal
data['label'] = data['path'].apply(lambda path: 'dog' if '/Dog' in path else
                                   ('cat' if '/Cat' in path else
                                    ('rabbit' if '/Rabbit' in path else
                                     ('fish' if '/Fish' in path else
                                      ('hamster' if '/Hamster' in path else 'mouse')))))

In [0]:
# Make a train-set split
train_data, test_data = data.random_split(0.8)

In [0]:
# Let's take a look at some training data
train_data.head()

In [0]:
# Train an image classification model - consider increasing max_iterations
model = tc.image_classifier.create(
    dataset=train_data,
    target='label',
    model='resnet-50',
    batch_size=16,
    max_iterations=10
)

# Image Classification Training Docs:
# https://apple.github.io/turicreate/docs/api/generated/turicreate.image_classifier.create.html#turicreate.image_classifier.create

In [0]:
# Let's see how the model performs on the hold out tes data
predictions = model.predict(test_data)
accuracy = tc.evaluation.accuracy(test_data['label'], predictions)
print(f"Image classifer is {accuracy*100} % accuracte on the testing dataset", flush=True)

## Model Export and Download

In [0]:
# Specify the CoreML model name
model_name = 'ImageClassifier'
coreml_model_name = model_name + '.mlmodel'

# Export the trained model to CoreML format
res = model.export_coreml(coreml_model_name)

In [0]:
# Use the file explorer to download the .mlmodel file you just created
# On the upper left side, expand the window, click the "Files" tab, right-click the file and select "download"